## **I. Importing Libraries**

In [1302]:
# Data Management Packages
import pandas as pd
import numpy as np
from collections import Counter

# Data Preprocessing Package
from sklearn.impute import KNNImputer

# Data Visualization packages
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Plotly Customization
template = "plotly_white"
px.defaults.template = "plotly_white"
color_discrete_sequence = px.colors.qualitative.Safe

# File system package
from os import listdir

In [1303]:
def read_CSV_in_years(YEARS = ('2019', '2020', '2021', '2022', '2023')):
    '''
    Parameters
    -----------
    YEARS        : Years to include/open.

    Parameters
    -----------
    Opens all the MRT train csv files within the provided years.
    Call cleanCSV() to open, clean, and return a dataframe.

    Return   
    ------------
    pd.Dataframe : All the cleaned CSVs compiled into a single dataframe.
    '''
    compiled_df = []
    FILES = listdir('MTR Parsed')

    # OPEN AND STORE ALL THE TARGET CSV FILES
    for fileName in FILES:
        if fileName[-4:] == '.csv' and fileName[:4] in YEARS:
            compiled_df.append(clean_CSV(fileName))
    
    return pd.concat(compiled_df)

In [1304]:
def clean_CSV(fileName, min_traffic_threshold = 0):
    '''
    Parameters
    -----------
    fileName               : String name of the .csv file.
    min_traffic_threshold  : Stations records with daily traffic below the threshold will be dropped.

    Parameters
    -----------
    Open the csv file and drop all records of stations with little/no activity for the entire day.

    Return   
    ------------
    pd.Dataframe : The cleaned csv data.
    '''
    df = pd.read_csv('MTR Parsed/' + fileName)
    # REMOVE INDEX COL
    df = df.drop(df.columns[0], axis=1)

    # CHECK THE TRAFFIC ACTIVITY OF PER STATION EVERYDAY 
    '''
    activity_check = df[['Day', 'Station_No', 'Net_Traffic']]
    activity_check = activity_check.groupby(['Day', 'Station_No']).sum()
    
    # KEEP ONLY THE STATION RECORDS THAT HAD ACTIVITY FOR THAT DAY
    records_to_keep = activity_check[activity_check.values > min_traffic_threshold]
    df = df.set_index(['Day', 'Station_No']).join(records_to_keep, lsuffix='' , rsuffix='_ActivityCheck')
    df = df.dropna(axis=0)
    df = df.rename(columns={df.columns[-1]:'Station_Total_Day_Traffic'})
    '''
    
    df['Date'] = pd.to_datetime(df['Date'])
    # df = df.reset_index()
    
    return df

## **II. Data Inspection**

In [1305]:
# Read csv using function
df = read_CSV_in_years()
# Reset the index of the dataframe because these are combined csv files, hence the duplication of indices
df = df.reset_index().drop(columns='index')
# Convert bool column to int
df['Is_Holiday'] = df.Is_Holiday.astype('int')
# Change Mon -> Sun range to 1 -> 7
df['Weekday'] += 1
# Dropping columns that are not needed
df.drop(columns=['Station_No', 'Exit', 'Net_Traffic'], inplace=True)

In [1306]:
# Display last 5 rows and information of the DataFrame
display(df.tail())
display(df.info())

,Date,Time,Station_Name,Entry,Year,Month,Day,Hour,Weekday,Is_Holiday
531084,2023-12-31,02:00 - 02:59,Guadalupe,0,2023,12,31,2,7,1
531085,2023-12-31,02:00 - 02:59,Buendia,0,2023,12,31,2,7,1
531086,2023-12-31,02:00 - 02:59,Ayala Ave,0,2023,12,31,2,7,1
531087,2023-12-31,02:00 - 02:59,Magallanes,0,2023,12,31,2,7,1
531088,2023-12-31,02:00 - 02:59,Taft,0,2023,12,31,2,7,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531089 entries, 0 to 531088
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          531089 non-null  datetime64[ns]
 1   Time          531089 non-null  object        
 2   Station_Name  531089 non-null  object        
 3   Entry         531089 non-null  int64         
 4   Year          531089 non-null  int64         
 5   Month         531089 non-null  int64         
 6   Day           531089 non-null  int64         
 7   Hour          531089 non-null  int64         
 8   Weekday       531089 non-null  int64         
 9   Is_Holiday    531089 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(6), object(2)
memory usage: 38.5+ MB


None

**Explanation:** It can be observed that there are no null values, however as we proceed with the analysis of the data, days without any ridership are identified. The entry ridership during operational hours on days without ridership will be transformed to NaN values, which will then be imputed.

## **III. Data Pre-processing**

### 3.1. Data Format Optimization

Since the numerical variables are within the int16 range and entry volume are whole numbers, it is safe to lower the int64 and float64 formats. 

In [1307]:
convertCols = df.select_dtypes(['int', 'float']).columns
df[convertCols].max()

for col in convertCols:
    if df[col].max() <= 127:
        df[col] = df[col].astype('int8')
    elif df[col].max() <= 32767:
        df[col] = df[col].astype('int16')
    else:
        print(f"{col} data out of range")
        assert False
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531089 entries, 0 to 531088
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          531089 non-null  datetime64[ns]
 1   Time          531089 non-null  object        
 2   Station_Name  531089 non-null  object        
 3   Entry         531089 non-null  int16         
 4   Year          531089 non-null  int16         
 5   Month         531089 non-null  int8          
 6   Day           531089 non-null  int8          
 7   Hour          531089 non-null  int8          
 8   Weekday       531089 non-null  int8          
 9   Is_Holiday    531089 non-null  int8          
dtypes: datetime64[ns](1), int16(2), int8(5), object(2)
memory usage: 16.7+ MB


**Explanation:** After lowering the formats of the numerical variables, the memory usage decreased from the original 38.5 MB to ~16.7MB

### 3.2. Converting DataFrame into a time series DataFrame format
The timeframe that was chosen was from April 18, 2022 to December 31, 2023, because this timeframe exemplified a more stable entry volume of passengers post-COVID.

In [1308]:
# Store post-COVID data from April 18, 2022 to December 31, 2023 from 4 am to 11pm (operational hours of MRT)
df_splice = df[(df.Date >= '2022-04-18') & ~(df.Hour.between(0,3))].copy()
df_splice = df_splice.reset_index().drop(columns='index')


display(df_splice.head())
display(df_splice.shape)

,Date,Time,Station_Name,Entry,Year,Month,Day,Hour,Weekday,Is_Holiday
0,2022-04-18,04:00 - 04:59,North Ave,566,2022,4,18,4,1,0
1,2022-04-18,04:00 - 04:59,Quezon Ave,603,2022,4,18,4,1,0
2,2022-04-18,04:00 - 04:59,GMA Kamuning,850,2022,4,18,4,1,0
3,2022-04-18,04:00 - 04:59,Cubao,527,2022,4,18,4,1,0
4,2022-04-18,04:00 - 04:59,Santolan,10,2022,4,18,4,1,0


(161980, 10)

In [1309]:
# Creating a disposable column to store integer Hour values
df_splice['Hour_delta'] = df_splice.Hour
# Converting the integer Hour values to timedelta format e.g. from 4 to 04:00:00
df_splice['Hour_delta'] = pd.to_timedelta(df_splice.Hour_delta, unit='h')
# Create Date_Time column to store the concatenated Date datetime values and Hour timedelta values
df_splice['Date_Time'] = df_splice.Date + df_splice.Hour_delta
# Rename the dataframe to df_ridership after creating a copy
df_ridership = df_splice.copy()
# Drop redundant columns
df_ridership = df_ridership.drop(['Date', 'Time', 'Hour_delta'], axis=1)
# Add Date_Time column as the first column and remove Date_Time column as the last column
df_ridership.insert(0, 'Date_Time', df_ridership.pop('Date_Time'))
# Set the Date_Time column as index to make the dataframe a time series data
df_ridership.set_index('Date_Time', inplace=True)

In [1310]:
# Display time series DataFrame and info
display(df_ridership)
display(df_ridership.info())

,Station_Name,Entry,Year,Month,Day,Hour,Weekday,Is_Holiday
Date_Time,,,,,,,,
2022-04-18 04:00:00,North Ave,566,2022,4,18,4,1,0
2022-04-18 04:00:00,Quezon Ave,603,2022,4,18,4,1,0
2022-04-18 04:00:00,GMA Kamuning,850,2022,4,18,4,1,0
2022-04-18 04:00:00,Cubao,527,2022,4,18,4,1,0
2022-04-18 04:00:00,Santolan,10,2022,4,18,4,1,0
...,...,...,...,...,...,...,...,...
2023-12-31 23:00:00,Guadalupe,0,2023,12,31,23,7,1
2023-12-31 23:00:00,Buendia,0,2023,12,31,23,7,1
2023-12-31 23:00:00,Ayala Ave,0,2023,12,31,23,7,1


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 161980 entries, 2022-04-18 04:00:00 to 2023-12-31 23:00:00
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Station_Name  161980 non-null  object
 1   Entry         161980 non-null  int16 
 2   Year          161980 non-null  int16 
 3   Month         161980 non-null  int8  
 4   Day           161980 non-null  int8  
 5   Hour          161980 non-null  int8  
 6   Weekday       161980 non-null  int8  
 7   Is_Holiday    161980 non-null  int8  
dtypes: int16(2), int8(5), object(1)
memory usage: 3.9+ MB


None

### 3.3. Creating maintenance days mask and integrating NaN data

Since our data has no null values, we have to differentiate which entry instances should be zero and null. It has been identified that MRT maintenance happened from April 06 to April 09 2023, hence it is accepetable to have zero ridership in these days. On the other hand, null values must be assigned to days with zero ridership during operational hours.

In [1311]:
# Sum the hourly entry volume within the day
all_stations_entry = df_ridership[['Entry']]
all_stations_daily = all_stations_entry.resample('D').sum()

# Maintenace start and end for 2023
maintenance_start_2023 = '2023-04-06'
maintenance_end_2023 = '2023-04-09'

# Create boolean masks for maintenance periods
maintenance_mask_2023 = (all_stations_daily.index >= maintenance_start_2023) & (all_stations_daily.index <= maintenance_end_2023)

# Find days with no ridership that are not in maintenance periods
no_ridership_days = all_stations_daily.index[(all_stations_daily['Entry'] == 0) & ~maintenance_mask_2023]

# From the hourly ridership DataFrame set the 0 values as NaN on days where there is no rdiership during operational hours
df_ridership.loc[df_ridership.index.normalize().isin(no_ridership_days), 'Entry'] = np.nan

In [1312]:
df_ridership.isna().sum()

Station_Name        0
Entry           49660
Year                0
Month               0
Day                 0
Hour                0
Weekday             0
Is_Holiday          0
dtype: int64

### 3.4. Data Imputation (KNN Imputation)

Among all time series imputation methods that were tried, such as the Kalman Smoothing Imputation, Multiple Imputation by Chained Equation, KNN Imputation, Linear and Polynomial imputation, the KNN imputation exhibited the best performance in imputing the missing values of the MRT ridership data as it mimicked the hourly seasonality patterns of the data.

In [1313]:
# Group data by station name
grouped_station = df_ridership.groupby('Station_Name')

# Store each station dataframe in a dictionary that can be accessed easily
station_dfs = {station: entry_data for station, entry_data in grouped_station}

# Initialize KNNImputer with 5 neighbors
knn_imputer = KNNImputer(n_neighbors=5)

# Dictionary to store imputed dataframes
imputed_station_dfs = {}

# Loop through each station dataframe and apply KNN Imputer
for station, df in station_dfs.items():
    # Make a copy of the dataframe
    df_copy = df.copy()
    # Drop the 'Station_Name' column
    station_name_column = df_copy.pop('Station_Name')
    # Apply KNNImputer
    imputed_data = knn_imputer.fit_transform(df_copy)
    # Store newly transformed DataFrame with imputed values in a DataFrame
    imputed_df = pd.DataFrame(data=imputed_data, columns=df_copy.columns, index=df_copy.index)
    # Add the 'Station_Name' column back to the dataframe
    imputed_df['Station_Name'] = station_name_column
    # Add the imputed dataframe to the dictionary
    imputed_station_dfs[station] = imputed_df

# Manually access the imputed dataframe for each station
north_ave_df_res_knn = imputed_station_dfs['North Ave']
quezon_ave_df_res_knn = imputed_station_dfs['Quezon Ave']
gma_kamuning_df_res_knn = imputed_station_dfs['GMA Kamuning']
cubao_df_res_knn = imputed_station_dfs['Cubao']
santolan_df_res_knn = imputed_station_dfs['Santolan']
ortigas_df_res_knn = imputed_station_dfs['Ortigas']
shaw_blvd_df_res_knn = imputed_station_dfs['Shaw Blvd']
boni_ave_df_res_knn = imputed_station_dfs['Boni Ave']
guadalupe_df_res_knn = imputed_station_dfs['Guadalupe']
buendia_df_res_knn = imputed_station_dfs['Buendia']
ayala_ave_df_res_knn = imputed_station_dfs['Ayala Ave']
magallanes_df_res_knn = imputed_station_dfs['Magallanes']
taft_df_res_knn = imputed_station_dfs['Taft']

In [1314]:
# Display North Ave DataFrame with the maintenance mask and integrated null values
print(f"Number of null values per feature:\n{station_dfs['North Ave'].isnull().sum()}")

Number of null values per feature:
Station_Name       0
Entry           3820
Year               0
Month              0
Day                0
Hour               0
Weekday            0
Is_Holiday         0
dtype: int64


*a.) Line Plot of Entry Ridership with Null Values*

In [1315]:
# Visualize the Hourly Entry Ridership with null values for North Avenue 
fig = px.line(station_dfs['North Ave'], y="Entry", title="<b>North Avenue Station Hourly Entry Ridership with Null Values</b><br>January 2022 to December 2023", color_discrete_sequence=color_discrete_sequence)

fig.show()

**Explanation:** It can be observed from the line plot that there are significant gaps in the data. We consider these entries, amounting to 3,820 instances, as null values.

*b.) Line Plot of Entry Ridership with Imputed Values*

In [1316]:
# Visualize the Hourly Entry Ridership with imputed values for North Avenue
# Initalize graph_objects figure
fig = go.Figure()

# Create line plot for the imputed North Avenue hourly entry ridership
fig.add_trace(go.Scatter(x=north_ave_df_res_knn.index, y=north_ave_df_res_knn.Entry,
                    mode='lines',
                    name='Imputed through KNN',
                    marker_color=color_discrete_sequence[0]))
# Create line plot for the original North Avenue hourly entry ridership
fig.add_trace(go.Scatter(x=station_dfs['North Ave'].index, y=station_dfs['North Ave'].Entry,
                    mode='lines',
                    name='Original plot',
                    marker_color=color_discrete_sequence[1]))
# Add title and integrate plotly template 
fig.update_layout(title="<b>North Avenue Station Hourly Entry Ridership</b><br>April 18 2022 to December 31 2023", template=template)

# Show figure
fig.show()

**Explanation:** Presented in the line plot is the imputed hourly entry ridership in North Avenue station. It can be observed that the behavior of the original plot is mimicked by the imputed values, which is a good indicator that the imputed values approximately follow the hourly seasonality patterns of the data. This also demonstrates that KNN imputation is an effective method for imputing wide gaps in time series data.